In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
import sys
from pathlib import Path
sys.path.append(Path(os.getcwd()).parent.parent.as_posix())

In [3]:
import dgl
import json
import optuna

import torch
import torch.nn as nn

from functools import partial
from torch.utils.data import Dataset, DataLoader

from tqdm import tqdm
from dataset import get_datasets, ETTDataset

from utils import seed_everything
from models.gcn import GCNModel

from constructor import construct_ess, construct_vanilla, construct_complete
from graph_features import spectral_features, deepwalk_features

from train import train_step, evaluation_step

import warnings
warnings.simplefilter("ignore")

/home/perkyfever/miniconda3/envs/project/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
seed_everything()

# Config

In [5]:
DATASET_NAME = "ETTh1.csv"

# LSTF setup
LOOKBACK_SIZE = 96
HORIZON_SIZE = 168

# Graphs setup
ALPHA = 0.05
GRAPH_CONSTRUCTION_FN = partial(construct_ess, alpha=ALPHA)
GRAPH_FEATURES_FN = partial(spectral_features, embed_size=7)

# Model setup
BATCH_SIZE = 64
HIDDEN_DIM = 256
NUM_LAYERS = 2
DROPOUT = 0.50
NUM_HEADS = 2
ACTIVATION_FN = nn.ReLU

# Train setup
NUM_EPOCHS = 20
LEARNING_RATE = 1e-3
WEIGHT_DECAY = 1e-5
PATIENCE = 2
LR_FACTOR = 0.5

# Dataset

In [6]:
train_ds, val_ds, test_ds = get_datasets(
    dataset_name="ETTh1.csv",
    lookback_size=LOOKBACK_SIZE,
    horizon_size=HORIZON_SIZE
)

In [7]:
class DatasetAdapter(Dataset):
    def __init__(self, dataset: ETTDataset, graph_construction_fn, graph_features_fn=None):
        super().__init__()
        self.graphs: list[dgl.DGLGraph] = []
        self.targets: list[torch.Tensor] = []
        for idx in tqdm(range(len(dataset)), desc="Building graphs"):
            x_data, time_data, y_data = dataset[idx]
            graph = graph_construction_fn(x_data)

            if graph_features_fn:
                graph_features = graph_features_fn(graph)
                graph.ndata["h"] = torch.cat([x_data.T, graph_features], dim=1)
            else:
                graph.ndata["h"] = x_data.T
            
            graph.ndata["h"] = torch.cat([
                graph.ndata["h"],
                time_data.repeat(graph.number_of_nodes(), 1),
            ], dim=1)
            
            self.targets.append(y_data)
            self.graphs.append(graph)

    def __len__(self):
        return len(self.graphs)

    def __getitem__(self, idx) -> tuple[dgl.DGLGraph, torch.Tensor]:
        return self.graphs[idx], self.targets[idx]

In [8]:
def graph_collate_fn(batch):
    """
    Custom collate function for batching DGL graphs.
    :param graphs: batch of graphs and targets
    :returns: batched graph, batch of targets
    """
    graphs, targets = zip(*batch)
    targets_tensor = torch.stack(targets, dim=0)
    return dgl.batch(graphs), targets_tensor

In [9]:
train_adapter_ds = DatasetAdapter(
    dataset=train_ds,
    graph_construction_fn=GRAPH_CONSTRUCTION_FN,
    graph_features_fn=GRAPH_FEATURES_FN
)

val_adapter_ds = DatasetAdapter(
    dataset=val_ds,
    graph_construction_fn=GRAPH_CONSTRUCTION_FN,
    graph_features_fn=GRAPH_FEATURES_FN
)

test_adapter_ds = DatasetAdapter(
    dataset=test_ds,
    graph_construction_fn=GRAPH_CONSTRUCTION_FN,
    graph_features_fn=GRAPH_FEATURES_FN
)

Building graphs: 100%|██████████| 2618/2618 [00:26<00:00, 98.52it/s] 


In [10]:
train_loader = DataLoader(
    dataset=train_adapter_ds,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=4,
    collate_fn=graph_collate_fn
)

val_loader = DataLoader(
    dataset=val_adapter_ds,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=4,
    collate_fn=graph_collate_fn
)

test_loader = DataLoader(
    dataset=test_adapter_ds,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=4,
    collate_fn=graph_collate_fn
)

In [11]:
INPUT_DIM = train_adapter_ds[0][0].ndata["h"].shape[1]
INPUT_DIM

178

# Model

In [12]:
class GraphTSModel(nn.Module):
    def __init__(
        self,
        input_dim: int,
        hidden_dim: int,
        num_layers: int,
        horizon_size: int,
        activation_fn: nn.Module,
        dropout: float = 0,
    ) -> "GraphTSModel":
        super().__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.activation_fn = activation_fn
        self.dropout = dropout
        self.horizon_size = horizon_size

        self.backbone = GCNModel(
            input_dim=self.input_dim,
            hidden_dim=self.hidden_dim,
            num_layers=self.num_layers,
            activation_fn=self.activation_fn,
            dropout=self.dropout
        )

        self.head = nn.Linear(self.hidden_dim, self.horizon_size)
    
    def forward(self, graph, features):
        x = features
        outputs = self.backbone(graph, x)
        tgt_emb = outputs[6::7] # extract OT's embeddings
        outputs = self.head(tgt_emb)
        return outputs

# Getters

In [13]:
def get_dataloaders(config) -> dict[str, DataLoader]:
    train_loader = DataLoader(
        dataset=config["train_ds"],
        batch_size=config["batch_size"],
        num_workers=4,
        shuffle=True,
        collate_fn=graph_collate_fn,
        drop_last=True
    )

    val_loader = DataLoader(
        dataset=config["val_ds"],
        batch_size=config["batch_size"],
        num_workers=4,
        shuffle=False,
        collate_fn=graph_collate_fn,
        drop_last=True
    )

    test_loader = DataLoader(
        dataset=config["test_ds"],
        batch_size=config["batch_size"],
        num_workers=4,
        shuffle=False,
        collate_fn=graph_collate_fn,
        drop_last=True
    )
    
    return {"train": train_loader, "val": val_loader, "test": test_loader}

In [14]:
def get_model(config) -> GraphTSModel:
    return GraphTSModel(
        input_dim=config["input_dim"],
        hidden_dim=config["hidden_dim"],
        num_layers=config["num_layers"],
        horizon_size=HORIZON_SIZE,
        activation_fn=ACTIVATION_FN,
        dropout=config["dropout"]
    )

In [15]:
def get_optimizer(model, config) -> torch.optim.Optimizer:
    return torch.optim.AdamW(
        model.parameters(), lr=config["lr"],
        weight_decay=config["weight_decay"]
    )

In [16]:
def get_scheduler(optimizer, config) -> torch.optim.lr_scheduler._LRScheduler:
    return torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer=optimizer, mode="min",
        factor=config["lr_factor"],
        patience=config["patience"]
    )

In [17]:
def get_criterion(config) -> nn.Module:
    return nn.MSELoss()

# Training

In [18]:
def run_experiment(config):    
    seed_everything()

    model = get_model(config)
    dataloaders = get_dataloaders(config)
    optimizer = get_optimizer(model, config)
    scheduler = get_scheduler(optimizer, config)
    criterion = get_criterion(config)
    
    report = {
        "train": [], "valid": [], "test": [],
        "score": 1e6, 'epoch': -1
    }
    
    global device
    model.to(device)

    pbar = tqdm(range(config["num_epochs"]))
    pbar.set_description("Training")
    
    for epoch in pbar:
        train_output = train_step(dataloaders["train"], model, optimizer, criterion, device)
        report["train"].append(train_output)

        validation_output = evaluation_step(dataloaders["val"], model, device)
        report["valid"].append(validation_output)

        test_output = evaluation_step(dataloaders["test"], model, device)
        report["test"].append(test_output)
        
        pbar.set_postfix_str(
            # f"[train] mse = {train_output['mse']:.4f} "
            # f"[train] mae = {train_output['mae']:.4f} "
            f"[valid] mse = {validation_output['mse']:.4f} "
            f"[valid] mae = {validation_output['mae']:.4f} "
            f"[test]  mse = {test_output['mse']:.4f} "
            f"[test]  mae = {test_output['mae']:.4f}"
        )

        if validation_output["mse"] < report["score"]:
            report["score"] = validation_output["mse"]
            report["epoch"] = epoch
        
        scheduler.step(validation_output["mse"])

    model.to("cpu")
    
    return report

In [19]:
def propose_config(config, trial):
    def propose_hparam_value(hparam_name, obj):
        hparam_value = obj
        if isinstance(obj, dict):
            distribution_type = obj["type"]
            distribution_kwargs = dict(filter(lambda p: p[0] != "type", obj.items()))
            suggest_fn = getattr(trial, f"suggest_{distribution_type}")
            hparam_value = suggest_fn(hparam_name, **distribution_kwargs)
        return hparam_value
    
    proposal_config = {}
    for hparam_name, obj in config.items():
        hparam_value = propose_hparam_value(hparam_name, obj)
        proposal_config[hparam_name] = hparam_value

    return proposal_config

def run_tuning(base_config):
    def objective(trial: optuna.Trial):
        proposal_config = propose_config(base_config, trial)
        # print(json.dumps(proposal_config, indent=4))
        experiment_report = run_experiment(proposal_config)
        sys.stdout.flush()
        return experiment_report["score"]

    study = optuna.create_study(
        direction="minimize",
        sampler=optuna.samplers.TPESampler(
            n_startup_trials=base_config["num_startup_trials"]
        )
    )
    study.optimize(objective, n_trials=base_config["num_trials"])
    return study

In [20]:
STUDY_RESULTS = []
HORIZON_SIZES = [24, 48, 168, 336, 720]

pbar = tqdm(HORIZON_SIZES)
for HORIZON_SIZE in pbar:
    pbar.set_description(f"Running tuning for horizon size {HORIZON_SIZE}")
    train_ds, val_ds, test_ds = get_datasets(
        dataset_name=DATASET_NAME,
        lookback_size=LOOKBACK_SIZE,
        horizon_size=HORIZON_SIZE
    )

    train_adapter_ds = DatasetAdapter(
        dataset=train_ds,
        graph_construction_fn=GRAPH_CONSTRUCTION_FN,
        graph_features_fn=GRAPH_FEATURES_FN
    )

    val_adapter_ds = DatasetAdapter(
        dataset=val_ds,
        graph_construction_fn=GRAPH_CONSTRUCTION_FN,
        graph_features_fn=GRAPH_FEATURES_FN
    )

    test_adapter_ds = DatasetAdapter(
        dataset=test_ds,
        graph_construction_fn=GRAPH_CONSTRUCTION_FN,
        graph_features_fn=GRAPH_FEATURES_FN
    )
    
    INPUT_DIM = train_adapter_ds[0][0].ndata["h"].shape[1]
    
    tuning_config = {
        "num_trials": 100,
        "num_startup_trials": 1,
        "train_ds": train_adapter_ds,
        "val_ds": val_adapter_ds,
        "test_ds": test_adapter_ds,
        "model": "GCN",
        "alpha": 0.05,
        "graph_strategy": "ess",
        "graph_features": "spectral",
        "lookback_size": LOOKBACK_SIZE,
        "horizon_size": HORIZON_SIZE,
        "time_usage": "None",
        "input_dim": INPUT_DIM,
        "lr_factor": 0.33,
        "patience": 2,
        "hidden_dim": {
            "type": "int",
            "low": 32,
            "high": 256,
            "step": 1,
            "log": True
        },
        "num_layers": {
            "type": "int",
            "low": 1,
            "high": 4,
            "step": 1,
        },
        "activation_fn": "ReLU",
        "batch_size": {
            "type": "int",
            "low": 32,
            "high": 256,
            "log": True
        },
        "num_epochs": 12,
        "weight_decay": 1e-5,
        "lr": {
            "type": "float",
            "low": 1e-4,
            "high": 1e-2,
            "log": True
        },
        "scheduler": "ReduceONPlateuau",
        "dropout": {
            "type": "float",
            "low": 0.0,
            "high": 0.35,
        },
    }
    
    study = run_tuning(tuning_config)
    STUDY_RESULTS.append(study)    

Building graphs: 100%|██████████| 2762/2762 [00:27<00:00, 102.13it/s]
[I 2025-04-26 15:52:51,681] A new study created in memory with name: no-name-b98b2f4a-1c26-49b2-ada2-d8a050ff9c16
Training: 100%|██████████| 12/12 [00:32<00:00,  2.69s/it, [valid] mse = 5.9031 [valid] mae = 1.8668 [test]  mse = 3.9157 [test]  mae = 1.4995]
[I 2025-04-26 15:53:24,573] Trial 0 finished with value: 5.84589991344105 and parameters: {'hidden_dim': 99, 'num_layers': 3, 'batch_size': 55, 'lr': 0.00019064261601508013, 'dropout': 0.025193650144129998}. Best is trial 0 with value: 5.84589991344105.
Training: 100%|██████████| 12/12 [00:34<00:00,  2.84s/it, [valid] mse = 6.0401 [valid] mae = 1.8927 [test]  mse = 3.7585 [test]  mae = 1.4714]
[I 2025-04-26 15:53:58,628] Trial 1 finished with value: 6.040124250023504 and parameters: {'hidden_dim': 106, 'num_layers': 3, 'batch_size': 52, 'lr': 0.00015291650356757003, 'dropout': 0.011920821238622259}. Best is trial 0 with value: 5.84589991344105.
Training: 100%|█████

In [21]:
import pickle

with open("gcn_study_results.pkl", "wb") as f:
    pickle.dump(STUDY_RESULTS, f)